In [28]:
!pip install nltk

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [29]:
import numpy as np
import nltk
from nltk.corpus import reuters 

In [43]:
cat_8 = ['acq', 'earn', 'grain', 'interest', 'money-fx']

def load_reuters(data_dir = None):
    nltk.download('reuters')
    
    dic_map = {}
    for i in range(len(cat_8)):
        dic_map[cat_8[i]] = i

    documents = reuters.fileids()
    train_docs = list(filter(lambda doc: doc.startswith("train"), documents));
    test_docs = list(filter(lambda doc: doc.startswith("test"), documents));
    
    def _get_data(docs):
        docs_text = []
        docs_label = []
        for index, i in  enumerate(docs):
            docs_text.append(reuters.raw(fileids=[i]))
            docs_label.append(reuters.categories(i))

        X = []
        y = []
        for cat in cat_8:
            count = 0
            for i in range(len(docs_label)): 
                if cat in docs_label[i]:
                    if len(docs_label[i]) > 1:
                        double = 0
                        for label in docs_label[i]:
                            if label in cat_8:
                                double += 1
                        if double > 1:
                            continue;
                    count += 1
                    X.append(docs_text[i])
                    y.append(cat)
        return X, y
    
    X_train, y_train = _get_data(train_docs)
    X_test, y_test = _get_data(test_docs)
    
    X = np.concatenate([X_train, X_test])
    y = np.concatenate([y_train, y_test])
    y = [dic_map[i] for i in y]
    
    return X, y

In [44]:
X, y = load_reuters()

[nltk_data] Downloading package reuters to /home/jovyan/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [53]:
print(X[70])

Woolworth Holdings says it bidding 244 mln stg for Superdrug stores

  Woolworth Holdings says it bidding 244 mln stg for Superdrug stores
  




In [45]:
len(y)

7694

In [46]:
from  collections import Counter
Counter(y)

Counter({0: 2347, 1: 3945, 2: 578, 3: 294, 4: 530})

In [34]:
from  collections import Counter
Counter(y)

Counter({0: 2319, 1: 458, 2: 3930, 3: 527, 4: 290, 5: 499, 6: 166, 7: 425})

In [25]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [26]:
data = pd.read_csv("./datasets/train.csv")

In [27]:
len(data)

159571

In [4]:
 doc_labels = (data['toxic'] == 1) | (data['severe_toxic']==1) | (data['obscene']==1) | (data['threat']==1) | (data['insult']==1) | (data['identity_hate']==1)

In [5]:
doc_labels

0         False
1         False
2         False
3         False
4         False
          ...  
159566    False
159567    False
159568    False
159569    False
159570    False
Length: 159571, dtype: bool

In [18]:
df = pd.DataFrame ({'id': data.id, 'comment_text': data.comment_text, 'label' : doc_labels.map({True: 1, False: 0})})
df = df.sample(frac=1, random_state=7).reset_index(drop=True)
df_sort = df.sort_values('label')
df_sort = df_sort.iloc[-50000: , :]

In [20]:
import re
def remove_IP(text):
    url = re.compile(r'[0-9]+(?:\.[0-9]+){3}')
    return url.sub(r'',text)

In [21]:
docs = df_sort['comment_text']
docs_label = df_sort['label']

NameError: name 'df_sort' is not defined

In [22]:
docs, docs_label = shuffle(docs, docs_label, random_state=42)

NameError: name 'docs' is not defined

In [21]:
from  collections import Counter

In [24]:
Counter(docs_label)

Counter({0: 33775, 1: 16225})

In [22]:
l = docs.tolist()

In [ ]:
[0-9]+(?:\.[0-9]+){3}

In [23]:
text = "Please note this is a mobile proxy IP address, and I believe it is fed through BIS, therefore it could potentially have half of Europe's BlackBerry connections through it . Please display a warning as necessary. 93.186.23.238"

In [24]:
remove_IP(text)

"Please note this is a mobile proxy IP address, and I believe it is fed through BIS, therefore it could potentially have half of Europe's BlackBerry connections through it . Please display a warning as necessary. "

In [23]:
for i in range(10):
    print(l[i])
    print('-----')

Numbering tables 

The problem with manually numbering lists of bridges is that new bridges are constructed all the time.  There are several long bridges that are under constructions that will force a renumbering of the entire list.  Your method, while a little less confusing, takes more work to maintain.  Are you willing to take on maintaining the re-numbering whenever someone adds a missing bridge?  I'm not sure it is worth the effort.
-----
"

 Oooo... 

Mr Administrator thinks he's all tough with his ""ZOMG-I-BLOCKS-YOU"" powers."
-----
Urdu Wikipedia 

Asalam-o-Alikum

I request you to please contribute to Urdu wikipedia. We are trying hard to build this wiki. Current article count is 617. Your help in any way would be appreciated.(even if you understand basic level of urdu)
-----
Please note this is a mobile proxy IP address, and I believe it is fed through BIS, therefore it could potentially have half of Europe's BlackBerry connections through it . Please display a warning as ne

In [59]:
from sklearn.utils import shuffle
import pandas as pd
import re
import numpy as np
import os
import unicodedata
from sklearn.datasets import fetch_20newsgroups

In [64]:
def _load_20news_groups():
    newsgroups_docs = fetch_20newsgroups(subset='all', shuffle=True, random_state=1)
    
    X = newsgroups_docs.data
    y = newsgroups_docs.target
    return X, y

In [65]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_email(text):
    url = re.compile(r'([A-Za-z0-9]*\.)*[A-Za-z0-9]*@([A-Za-z]*\.?[A-Za-z0-9]*)*')
    return url.sub(r'',text)

In [66]:
X, y = _load_20news_groups()

In [69]:
for i in range(len(X)):
            u = unicodedata.normalize('NFKD', X[i]).encode('ascii','ignore')
            for character in [b"\n", b"|", b">", b"<", b"-", b"+", b"^", b"[", b"]", b"#", b"\t", b"\r", b"`", b'_']:
                u = u.replace(character, b" ")
                s = str(u, 'utf8')
                s = remove_URL(s)
                s = remove_email(s)
                X[i] = s

In [71]:
X[0]

'From:  (Paul Harvey) Subject: Re: Clarification of personal position Organization: The Duck Pond public unix:  1 408 249 9630, log in as \'guest\'. Lines: 26  In article      (Paul Hudson Jr) writes:  In article      (Darius Lecointe) writes:   If it were a sin to violate Sunday no one could   ever be forgiven for that for Jesus never kept Sunday holy.  He only   recognized one day of the seven as holy.  Jesus also recognized other holy days, like the Passover.  Acts 15 says   that no more should be layed on the Gentiles than that which is necessary.  The sabbath is not in the list, nor do any of the epistles instruct people  to keep the 7th day, while Christians were living among people who did not  keep the 7th day.  It looks like that would have been a problem.  Instead, we have Scriptures telling us that all days can be esteemed alike  (Romans 14:5) and that no man should judge us in regard to what kind of  food we eat, Jewish holy days we keep, or  in regard to the sabbath. (Col.

In [21]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_docs = fetch_20newsgroups(subset='all', shuffle=True, random_state=1)

In [22]:
X = newsgroups_docs.data
y = newsgroups_docs.target

In [27]:
import unicodedata

In [29]:
u = unicodedata.normalize('NFKD', X[0]).encode('ascii','ignore')

In [37]:
for character in [b"\n", b"|", b">", b"<", b"-", b"+", b"^", b"[", b"]", b"#", b"\t", b"\r", b"`", b'_']:
    u = u.replace(character, b" ")

In [56]:
import re
# to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_email(text):
    url = re.compile(r'([A-Za-z0-9]*\.)*[A-Za-z0-9]*@([A-Za-z]*\.?[A-Za-z0-9]*)*')
    return url.sub(r'',text)

# to remove html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [57]:
s = str(u, 'utf8')
s

'From: pharvey@quack.kfu.com (Paul Harvey) Subject: Re: Clarification of personal position Organization: The Duck Pond public unix:  1 408 249 9630, log in as \'guest\'. Lines: 26  In article  C5rBHt.Fw4@athena.cs.uga.edu   hudson@athena.cs.uga.edu (Paul Hudson Jr) writes:  In article  C5MuIw.AqC@mailer.cc.fsu.edu   dlecoint@garnet.acns.fsu.edu (Darius Lecointe) writes:   If it were a sin to violate Sunday no one could   ever be forgiven for that for Jesus never kept Sunday holy.  He only   recognized one day of the seven as holy.  Jesus also recognized other holy days, like the Passover.  Acts 15 says   that no more should be layed on the Gentiles than that which is necessary.  The sabbath is not in the list, nor do any of the epistles instruct people  to keep the 7th day, while Christians were living among people who did not  keep the 7th day.  It looks like that would have been a problem.  Instead, we have Scriptures telling us that all days can be esteemed alike  (Romans 14:5) and 

In [58]:
remove_email(s)

'From:  (Paul Harvey) Subject: Re: Clarification of personal position Organization: The Duck Pond public unix:  1 408 249 9630, log in as \'guest\'. Lines: 26  In article      (Paul Hudson Jr) writes:  In article      (Darius Lecointe) writes:   If it were a sin to violate Sunday no one could   ever be forgiven for that for Jesus never kept Sunday holy.  He only   recognized one day of the seven as holy.  Jesus also recognized other holy days, like the Passover.  Acts 15 says   that no more should be layed on the Gentiles than that which is necessary.  The sabbath is not in the list, nor do any of the epistles instruct people  to keep the 7th day, while Christians were living among people who did not  keep the 7th day.  It looks like that would have been a problem.  Instead, we have Scriptures telling us that all days can be esteemed alike  (Romans 14:5) and that no man should judge us in regard to what kind of  food we eat, Jewish holy days we keep, or  in regard to the sabbath. (Col.

In [2]:
import pandas as pd
import re
import numpy as np
import os

In [3]:
def _load_app_store(data_dir = './dataset.csv'):
    df = pd.read_csv(data_dir)
    
    df_en = df[(df.lang == 'en') & (df.source == 'app_review') & ~df['text'].isna()]
    
    X = df_en['text'].to_list()
    y = df_en['category'].map({'inq': 0, 'pbr': 1, 'irr': 2}).to_list()
    
    return X, y

In [4]:
X, y = _load_app_store()

In [1]:
## IMDB

In [11]:
import pandas as pd
import re
import numpy as np

In [12]:
import os

def _load_imdb(data_dir = '/home/jovyan/Active Learning/data/aclImdb/'):
    X = []
    y = []
    for partition in ["train", "test"]:
        for category  in ["pos", "neg"]:
            lable = 0 if category  == "neg" else 1

            path = os.path.join(data_dir, partition, category )
            files = os.listdir(path)
            for f_name in files:
                with open(os.path.join(path, f_name), "r") as f:
                    review = f.read()
                    X.append(review)
                    y.append(lable)

    return X, y

X, y = _load_imdb()

In [13]:
# to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# to remove html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [14]:
df = pd.DataFrame(data=(X))
df.head()

,0
0,Although I have not seen this mini-series in o...
1,In terms of visual beauty this movie is outsta...
2,"Meatball Machine is an amazing splatter film, ..."
3,I have watched this movie three times. The las...
4,"Okay, so the first few seasons took a while to..."


In [15]:
df[0] = df[0].apply(remove_URL)
df[0] = df[0].apply(remove_html)

In [16]:
np.save('imdb_X', df[0].tolist())

In [17]:
np.save('imdb_y', y)

In [19]:
import os.path
os.path.isfile('./imdb_X.npy') 

True